### Importing Data and Necessary libraries

In [1]:
import pandas as pd
import sqlite3
import csv

In [2]:
!pip install ipython-sql

### Importing csv files using Python

In [3]:
# Importing and converting cvs data files into dataframes  
df1 = pd.read_csv('HRData_Diversity.csv')
df2 = pd.read_csv('HRData_Employees.csv')
df3 = pd.read_csv('HRData_Manager.csv')
df4 = pd.read_csv('HRData_Positions.csv')

In [4]:
# Connecting to database 
conn = sqlite3.connect('hr_data_database.db')

In [5]:
# Adding dataframes to SQL database to use for analysis
df1.to_sql('HR_Diversity', conn, index=False,)
df2.to_sql('HR_Employees', conn, index=False)
df3.to_sql('HR_Manager', conn, index=False)
df4.to_sql('HR_Positions', conn, index=False)

29

In [6]:
# Load the SQL extension to execute SQL commands in this notebook cell
%load_ext sql

In [7]:
# Connect to the SQLite database named 'hr_database.db'
%sql sqlite:///hr_data_database.db

### Exploring the Data in our Database
- Understand the schema of our four tables
- Identify Primary and foreign keys in our tables

In [8]:
%%sql

-- Retrieve the names of tables in the SQLite database
SELECT name AS hr_database_tables
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///hr_data_database.db
Done.


hr_database_tables
HR_Diversity
HR_Employees
HR_Manager
HR_Positions


In [9]:
%%sql

-- Show the schema of the table HR_Employees
PRAGMA table_info(HR_Employees);

 * sqlite:///hr_data_database.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Employee_Name,TEXT,0,None,0
1,EmpID,INTEGER,0,None,0
2,DeptID,INTEGER,0,None,0
3,Salary,INTEGER,0,None,0
4,PositionID,INTEGER,0,None,0
5,State,TEXT,0,None,0
6,DOB,TEXT,0,None,0
7,MaritalDesc,TEXT,0,None,0
8,DateofHire,TEXT,0,None,0
9,DateofTermination,TEXT,0,None,0


In [10]:
%%sql

-- Show the schema of the table HR_Positions
PRAGMA table_info(HR_Positions);

 * sqlite:///hr_data_database.db
Done.


cid,name,type,notnull,dflt_value,pk
0,PositionID,INTEGER,0,None,0
1,Position,TEXT,0,None,0


In [11]:
%%sql

-- Show the schema of the table
PRAGMA table_info(HR_Manager);

 * sqlite:///hr_data_database.db
Done.


cid,name,type,notnull,dflt_value,pk
0,EmpID,INTEGER,0,None,0
1,ManagerName,TEXT,0,None,0
2,ManagerID,REAL,0,None,0


In [12]:
%%sql

-- Show the schema of the table
PRAGMA table_info(HR_Diversity);

 * sqlite:///hr_data_database.db
Done.


cid,name,type,notnull,dflt_value,pk
0,EmpID,INTEGER,0,None,0
1,Sex,TEXT,0,None,0
2,FromDiversityJobFairID,TEXT,0,None,0
3,HispanicLatino,TEXT,0,None,0
4,RaceDesc,TEXT,0,None,0


### Data Cleaning - Create a new table named HR_Dataset by combining data from multiple tables

In [13]:
%%sql

-- Create a new table named HR_Dataset by combining data from multiple tables
CREATE TABLE HR_Dataset AS

SELECT *
FROM HR_Employees e
inner JOIN HR_Positions p ON e.positionID = p.positionID
inner JOIN HR_Manager m on e.empid = m.empid
inner JOIN HR_Diversity d on e.empid = d.empid;

 * sqlite:///hr_data_database.db
Done.


[]

In [15]:
%%sql 
--
Select * from HR_Dataset limit 1;

 * sqlite:///hr_data_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White


### Ensuring new HR_Dataset has been added to database

In [14]:
%%sql

-- Retrieve the names of tables in the SQLite database
SELECT name AS hr_database_tables
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///hr_data_database.db
Done.


hr_database_tables
HR_Diversity
HR_Employees
HR_Manager
HR_Positions
HR_Dataset


### Data Cleaning - Dropping Duplicate Columns from HR_Dataset table
- Remove Redundant Columns from HR_Dataset Table that were added when joining the 4 tables in the database

In [54]:
%%sql 

-- Remove the column 'positionid:1' from the HR_Dataset table
ALTER TABLE HR_Dataset
DROP COLUMN 'positionid:1';

-- Remove the column 'empid:1' from the HR_Dataset table
ALTER TABLE HR_Dataset
DROP COLUMN 'empid:1';

-- Remove the column 'managerid:1' from the HR_Dataset table
ALTER TABLE HR_Dataset
DROP COLUMN 'managerid:1';

-- Remove the column 'empid:2' from the HR_Dataset table
ALTER TABLE HR_Dataset
DROP COLUMN 'empid:2';

 * sqlite:///hr_database.db
   sqlite:///my_database.db
Done.
Done.
Done.
Done.


[]

In [55]:
%%sql 
-- Display the modified table
select * from hr_dataset limit 1;

 * sqlite:///hr_database.db
   sqlite:///my_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,Position,ManagerName,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,Production Technician I,Michael Albert,M,No,No,White


### Data Cleaning - Changing Data Types

- Updating data type from text to date for DOB, DateofHire, and DateofTermination columns

In [16]:
%%sql

-- Add a new column 'date_DOB' to the table and update column to a date data type 
ALTER TABLE HR_Dataset ADD COLUMN date_DOB date;

UPDATE HR_Dataset
SET date_DOB = strftime('%Y-%m-%d', DOB);


-- Add a new column 'date_hired' to the table and update column to a date data type
ALTER TABLE HR_Dataset ADD COLUMN date_hired date;

UPDATE HR_Dataset
SET date_hired = strftime('%Y-%m-%d', dateofhire);


-- Add a new column 'date_term' to the table and update column to a date data type
ALTER TABLE HR_Dataset ADD COLUMN date_term date;

UPDATE HR_Dataset
SET date_term = strftime('%Y-%m-%d', dateoftermination);


-- Display the modified table with three added columns
SELECT * FROM HR_Dataset

LIMIT 1;

 * sqlite:///hr_data_database.db
Done.
311 rows affected.
Done.
311 rows affected.
Done.
311 rows affected.
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White,1983-07-10,2011-07-05,None


### Data Manipulation - Replacing Null values from date of termination
- Replacing all Null values from the date_term column as current date to create a column for display employee tenure

In [17]:
%%sql

UPDATE hr_dataset

SET date_term = CURRENT_DATE
WHERE date_term IS NULL;

 * sqlite:///hr_data_database.db
207 rows affected.


[]

In [18]:
%%sql 

-- Display the modified table
SELECT * FROM HR_Dataset LIMIT 1;

 * sqlite:///hr_data_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White,1983-07-10,2011-07-05,2023-08-10


### Data Manipulation - Adding Employee Tenure Column

In [19]:
%%sql 

ALTER TABLE HR_Dataset
ADD COLUMN Tenure INTEGER;

UPDATE HR_Dataset
SET Tenure = CAST((julianday(date_term) - julianday(date_hired)) / 365.25 AS INTEGER);

 * sqlite:///hr_data_database.db
Done.
311 rows affected.


[]

In [20]:
%%sql 

-- Display the modified table
SELECT * FROM HR_Dataset LIMIT 1;

 * sqlite:///hr_data_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term,Tenure
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White,1983-07-10,2011-07-05,2023-08-10,12


### Data Manipulation
- Added a new column 'Married_id' to the HR_Employees table 
- Update 'Married_id' values based on 'MaritalDesc' values

In [21]:
%%sql

ALTER TABLE HR_Dataset
ADD COLUMN Married_id INTEGER;

UPDATE HR_Dataset
SET Married_id = CASE
    WHEN MaritalDesc = 'Single' THEN 1
    WHEN MaritalDesc = 'Widowed' THEN 1
    WHEN MaritalDesc = 'Divorced' THEN 1
    WHEN MaritalDesc = 'Married' THEN 0
    ELSE NULL
END;

 * sqlite:///hr_data_database.db
Done.
311 rows affected.


[]

In [27]:
%%sql 

-- Display the modified table
SELECT * FROM HR_Dataset LIMIT 1;

 * sqlite:///hr_data_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term,Tenure,Married_id,PerfScore_ID,DiversityJobFairScore
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White,1983-07-10,2011-07-05,2023-08-10,12,1,4,0


### Data Manipulation
- Added a new column 'PerfScore_ID' to the HR_Employees table 
- Update 'PerfScore_ID' values based on 'PerformanceScore' values

In [23]:
%%sql

ALTER TABLE HR_Dataset
ADD COLUMN PerfScore_ID INTEGER;

UPDATE HR_Dataset
SET PerfScore_ID = CASE
    WHEN PerformanceScore = 'Exceeds' THEN 4
    WHEN PerformanceScore = 'Fully Meets' THEN 3
    WHEN PerformanceScore = 'Needs Improvement' THEN 2
    WHEN PerformanceScore = 'PIP' THEN 1
    ELSE NULL
END;

 * sqlite:///hr_data_database.db
Done.
311 rows affected.


[]

In [78]:
%%sql 

-- Display the modified table
SELECT * FROM HR_Dataset LIMIT 1;

 * sqlite:///hr_database.db
   sqlite:///my_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,Position,ManagerName,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term,Tenure,PerfScore_ID
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,Production Technician I,Michael Albert,M,No,No,White,1983-07-10,2011-07-05,2023-08-10,12,4


### Data Manipulation
- Added a new column 'DiversityJobFairScore' to the HR_Dataset table 
- Updated score values as 1 if employees were hired in a Diversity Job Fair and 0 if not 

In [24]:
%%sql 

ALTER TABLE HR_Dataset
ADD COLUMN DiversityJobFairScore INTEGER;

UPDATE HR_Dataset
SET DiversityJobFairScore = CASE
    WHEN FromDiversityJobFairID = 'Yes' THEN 1
    ELSE 0
END;

 * sqlite:///hr_data_database.db
Done.
311 rows affected.


[]

In [25]:
%%sql 

-- Display the modified table
SELECT * FROM HR_Dataset LIMIT 1;

 * sqlite:///hr_data_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term,Tenure,Married_id,PerfScore_ID,DiversityJobFairScore
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White,1983-07-10,2011-07-05,2023-08-10,12,1,4,0


### Data Manipulation
- Added a new column 'Date_Term' to the HR_Dataset table 
- Update values based on 'dateoftermination' column, 1 if employee had a termination date and 0 if date was Null

In [28]:
%%sql 

UPDATE HR_Dataset
SET date_term = CASE
    WHEN dateoftermination IS NULL THEN 0 
    ELSE 1
END;

 * sqlite:///hr_data_database.db
311 rows affected.


[]

In [25]:
%%sql 

-- Display the modified table
SELECT * FROM HR_Dataset LIMIT 1;

 * sqlite:///hr_data_database.db
Done.


Employee_Name,EmpID,DeptID,Salary,PositionID,State,DOB,MaritalDesc,DateofHire,DateofTermination,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,Absences,PositionID:1,Position,EmpID:1,ManagerName,ManagerID:1,EmpID:2,Sex,FromDiversityJobFairID,HispanicLatino,RaceDesc,date_DOB,date_hired,date_term,Tenure,Married_id,PerfScore_ID,DiversityJobFairScore
"Adinolfi, Wilson K",10026,5,62506,19,MA,1983-07-10,Single,2011-07-05,None,22.0,LinkedIn,Exceeds,4.6,5,0,1,19,Production Technician I,10026,Michael Albert,22.0,10026,M,No,No,White,1983-07-10,2011-07-05,2023-08-10,12,1,4,0


### Export clean HR_Dataset table as a CVS file

In [22]:
# Query the hr_dataset table using the %sql magic command
result = %sql SELECT * FROM hr_dataset

# Convert the result to a DataFrame
df = result.DataFrame()

# Specify the CSV file path
csv_file_path = 'hr_dataset.csv'

# Export the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

 * sqlite:///hr_database.db
Done.
